<a href="https://colab.research.google.com/github/K-107/BERTsDay/blob/main/BertTrainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# GPU 정보 보기
! nvidia-smi --query | fgrep 'Product Name'

    Product Name                          : Tesla P100-PCIE-16GB


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# 리포지토리 복제
! git clone https://github.com/K-107/BERTsDay.git

Cloning into 'BERTsDay'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 58 (delta 11), reused 27 (delta 4), pack-reused 0
Unpacking objects: 100% (58/58), done.


In [5]:
# 밑의 모듈은 텐서플로 1.x로 작성되었으므로 버전 변경
! pip install --upgrade tensorflow==1.15.0rc3

     |████████████████████████████████| 412.3MB 27kB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
     |████████████████████████████████| 3.8MB 25.6MB/s 
     |████████████████████████████████| 512kB 34.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=5e9f182c5e4030e994d7f30b247aa8034dc6ee618d577ec7c97e9dd74de86f18
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tens

## pretrained BERT 모델을 모듈로 export

ETRI에서 사전훈련한 BERT의 체크포인트를 가지고 BERT 모듈을 만드는 과정.

In [ ]:
! python /content/BERTsDay/Bert_fine_tuning/export_korbert/bert_to_module.py -i "/content/drive/MyDrive/3_Slot_Tagging_Proj/004_bert_eojeol_tensorflow" -o "/content/drive/MyDrive/3_Slot_Tagging_Proj"

## 데이터 준비

모델을 훈련하기 위해 필요한 seq.in, seq.out이라는 2가지 파일을 만드는 과정.



In [5]:
%cd /content/drive/MyDrive/3_Slot_Tagging_Proj/
#!python make_data.py train_data.txt > train_data.txt

/content/drive/MyDrive/3_Slot_Tagging_Proj


In [ ]:
#1. 텍스트 파일에서 랜덤한 문장만 추출하는 명령어
!shuf -n 1000000 ./output_sample2.txt > ./sample/temp_0.txt

In [ ]:
# 2. 공백 문자 여러개를 하나로 줄이는 명령어
! sed 's/  */ /g' ./sample/temp_0.txt > ./sample/temp_1.txt

In [ ]:
#3. 앞, 뒤의 공백 문자를 없애는 명렁어
#앞 :
! sed 's/^ //g' ./sample/temp_1.txt > ./sample/temp_2.txt
#뒤 :
! sed 's/ $//g' ./sample/temp_2.txt > ./sample/temp_3.txt

In [ ]:
#4. 중복 제거하는 명령어 (두 가지 방법 모두 동일합니다.)
! sort -u ./sample/temp_3.txt > ./sample/sample_1M.txt

# 임시 파일들 제거
! rm ./sample/temp_0.txt
! rm ./sample/temp_1.txt
! rm ./sample/temp_2.txt
! rm ./sample/temp_3.txt

tcmalloc: large alloc 1706696704 bytes == 0x55df4a29c000 @  0x7efd9176f1e7 0x55df497fc718 0x55df497fb5a1 0x7efd9114dbf7 0x55df497fc02a


In [ ]:
# 실행 전에 소스 수정 필요함
! python /content/BERTsDay/Bert_fine_tuning/prepare_data.py -i ./sample/sample_1M.txt -o ./sample/

### Fine-tuing 훈련

어떻게 하면 train_bert_finetuning.py 코드를 실행할 수 있는지 코드 내부의 parser을 참조하여 작성하세요.

In [13]:
# 파인튜닝 훈련 모듈의 도움말
! python /content/BERTsDay/Bert_fine_tuning/train_bert_finetuning.py --help

usage: Training the BERT NLU model [-h] --train TRAIN [--val VAL] --save SAVE
                                   [--epochs EPOCHS] [--batch BATCH]
                                   [--type TYPE] [--bertpath BERTPATH]

optional arguments:
  -h, --help            show this help message and exit
  --train TRAIN, -t TRAIN
                        Path to training set
  --val VAL, -v VAL     Path to validation set
  --save SAVE, -s SAVE  Folder path to save the trained model
  --epochs EPOCHS, -e EPOCHS
                        Number of epochs
  --batch BATCH, -bs BATCH
                        Batch size
  --type TYPE, -tp TYPE
                        bert or albert
  --bertpath BERTPATH, -bp BERTPATH
                        bert model hub path (=modularized pretrained bert
                        path)


In [6]:
%cd /content/drive/MyDrive/3_Slot_Tagging_Proj/
! python /content/BERTsDay/Bert_fine_tuning/train_bert_finetuning.py -bp "/content/drive/MyDrive/3_Slot_Tagging_Proj/Bert_pretrained" -t "./sample/" -s "./Fine_tuned" -e 3 -bs 256 -tp bert


2021-04-13 02:51:09.217821: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-04-13 02:51:09.222510: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2021-04-13 02:51:09.222709: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55afbca80680 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-04-13 02:51:09.222739: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-04-13 02:51:09.226727: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-04-13 02:51:09.410315: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-04-13 02:51:09.411152:

## 모델 평가

TODO - 위의 내용처럼 어떻게 하면 eval_bert_finetuned.py 코드를 실행할 수 있는지 코드 내부의 parser을 참조하여 작성하세요.
테스트의 결과는 --model에 넣어준 모델 경로 아래의 test_results에 저장된다.

In [12]:
# 파인튜닝 모델 평가 모듈의 도움말
! python /content/BERTsDay/Bert_fine_tuning/eval_bert_finetuned.py --help

usage: Evaluating the BERT / ALBERT NLU model [-h] --model MODEL --data DATA
                                              [--type TYPE]
                                              [--bertpath BERTPATH]

optional arguments:
  -h, --help            show this help message and exit
  --model MODEL, -m MODEL
                        Path to BERT / ALBERT NLU model
  --data DATA, -d DATA  Path to data
  --type TYPE, -tp TYPE
                        bert or albert
  --bertpath BERTPATH, -bp BERTPATH
                        bert model hub path (=modularized pretrained bert
                        path)


In [9]:
%cd /content/drive/MyDrive/3_Slot_Tagging_Proj/
! python /content/BERTsDay/Bert_fine_tuning/eval_bert_finetuned.py -bp "./Bert_pretrained/" -t bert -m "./Fine_tuned/" -d "./sample/"

/content/drive/MyDrive/3_Slot_Tagging_Proj


2021-04-13 07:46:07.962338: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-04-13 07:46:07.968568: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2021-04-13 07:46:07.969076: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x556ab1b7c680 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-04-13 07:46:07.969116: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-04-13 07:46:07.971113: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-04-13 07:46:07.975677: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-04-13 07:46:07.975740: I tensorflow/

## Inference (임의의 문장을 모델에 넣어보기)

TODO - eval_bert_finetuned.py를 참고하여 한 문장씩 넣어서 모델이 내뱉는 결과물을 볼 수 있도록 inference.py 코드를 완성하세요.  
python inference.py --model {훈련된 모델이 저장된 경로}  
예시: python inference.py --model saved_model/  
모델 자체가 용량이 커서 불러오는 데까지 시간이 걸림  
"Enter your sentence:"라는 문구가 나오면 모델에 넣어보고 싶은 문장을 넣어 주면 됨  
quit라는 입력을 넣어 주면 종료  

In [11]:
# 파인튜닝 추론 모듈의 도움말
! python /content/BERTsDay/Bert_fine_tuning/inference.py --help

usage: Evaluating the BERT / ALBERT NLU model [-h] --model MODEL [--type TYPE]
                                              [--bertpath BERTPATH]

optional arguments:
  -h, --help            show this help message and exit
  --model MODEL, -m MODEL
                        Path to BERT / ALBERT NLU model
  --type TYPE, -tp TYPE
                        bert or albert
  --bertpath BERTPATH, -bp BERTPATH
                        bert model hub path (=modularized pretrained bert
                        path)


In [10]:
# 
%cd /content/drive/MyDrive/3_Slot_Tagging_Proj/
!python /content/BERTsDay/Bert_fine_tuning/inference.py -bp "./Bert_pretrained/" -m "./Fine_tuned" -tp bert

/content/drive/MyDrive/3_Slot_Tagging_Proj

2021-04-13 07:48:27.703715: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-04-13 07:48:27.708943: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2021-04-13 07:48:27.709135: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5558ece24680 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-04-13 07:48:27.709174: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-04-13 07:48:27.711074: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-04-13 07:48:27.715087: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-04-13 07:48:27.715134: I tensorflow/s